In [1]:
from pathlib import Path
import math
import gdstk
import gdsfactory as gf
from qnlfactory import *
import numpy as np
from gdsfactory.cross_section import ComponentAlongPath
import qnlmodels.cpw as CPW
import qnlmodels.klopfenstein_taper as kt
from qnlfactory.components import (border, 
                                   launch, 
                                   solder_bump_array, 
                                   flip_chip_alignment,
                                   twpa_cell,
                                   titanium_array)

layermap = QFaBLayers

In [2]:
def round_to_even_float(x):
    return round(x/2,3)*2

def kerr_twpa(
        bottom_chip_size: tuple=(10_000, 10_000), 
        top_chip_size: tuple=(9_000, 9_000), 
        pad_size: tuple = (350, 100),
        finger_size: tuple = (40, 50),
        num_fingers: int = 4,
        finger_gap: float = 40,
        etch_padding: float = 4,
        taper_length: float = 90,
        snail_frame_size: tuple = (30, 14),
        loop_size: tuple = (10,10),
        dolan_bridge_width: float = 0.2,
        large_bridge_height: float = 3,
        large_bridge_gap: float = 0.5,
        junction_ratio: float = 0.1,
        large_junction_taper_length: float = 0.5,
        small_junction_taper_length: float = 0.75,
        small_junction_lead_length: float = 2,
        cell_impedance: float = 50,
        margin: float = 250,
        segment_spacing: float = 150, 
        launch_size: tuple = (250, 250),
        bump_diameter: float = 50,
        underbump_metal_width: float = 80):
    
    # Substrate parameters 
    chip_width, chip_height = bottom_chip_size
    top_chip_width, top_chip_height = top_chip_size
    finger_width, finger_length = finger_size

    # Input geometry
    ## Starting location
    input_loc = (-chip_width/2+margin+launch_size[0]/2, 0)
    
    amp_cpw = CPW.CPW(snail_frame_size[1], finger_width, 675, 11.7)
    amp_cpw = CPW.CPW(snail_frame_size[1], amp_cpw.solve_for_impedance(cell_impedance,'s'), 675, 11.7)
    cpw_50_ohms = CPW.CPW(amp_cpw.solve_for_impedance(cell_impedance,'w'), amp_cpw.solve_for_impedance(cell_impedance,'s'), 675, 11.7)

    twpa = gf.Component()
    cell_width, cell_height = (pad_size[0]+2*taper_length+snail_frame_size[0], pad_size[1]+2*finger_size[1])
    
    cell_array = gf.Component()
    N_cells_per_row = math.floor((top_chip_height)/2/cell_width)
    M_rows = math.floor((top_chip_width)/(cell_height+segment_spacing))
    center_array = (-cell_width*N_cells_per_row+cell_width/2+snail_frame_size[0]/2, -(cell_height+segment_spacing)*(M_rows-1)/2)             
    
    cell = twpa_cell(input_cpw_spacing=cpw_50_ohms.s, 
                     pad_size=pad_size,
                     finger_size=finger_size,
                     num_fingers=num_fingers,
                     finger_gap=finger_gap,
                     finger_padding=etch_padding,
                     taper_length=taper_length,
                     loop_size=loop_size,
                     dolan_bridge_width=dolan_bridge_width,
                     large_bridge_height=large_bridge_height,
                     large_bridge_gap=large_bridge_gap,
                     junction_ratio=junction_ratio,
                     large_junction_taper_length=large_junction_taper_length,
                     small_junction_taper_length=small_junction_taper_length,
                     small_junction_lead_length=small_junction_lead_length
                     )
    alt_cells = gf.Component()
    alt_cells.add_ref(cell)
    alt_cells.add_ref(cell).mirror_y().dmovex(cell_width)
    

    cell_array.add_ref(alt_cells, columns=N_cells_per_row, rows=M_rows/2+1, spacing=(2*cell_width, 2*(cell_height+segment_spacing))).dmove(center_array).drotate(angle=-90, center=(0,0))
    cell_array.add_ref(alt_cells, columns=N_cells_per_row, rows=M_rows/2, spacing=2*(2*cell_width, 2*(cell_height+segment_spacing))).dmove(center_array).drotate(angle=-270, center=(0,0)).dmovex(-cell_height-segment_spacing)

    twpa.add_ref(cell_array)
    print(f"{N_cells_per_row*(M_rows)} cells") 

    ## Impedance matched trace
    input_trace = Trace(width=cpw_50_ohms.w, spacing=cpw_50_ohms.s)
    
    chain_start = (-(cell_height+segment_spacing)*(M_rows-1)/2,-cell_width*N_cells_per_row)
    turn_radius = segment_spacing/2 + cell_height/2
    io_turn_radius = chain_start[0] - 2*turn_radius  - input_loc[0]
    io_length = input_loc[1] - io_turn_radius - chain_start[1]
    
    ## Input -- CPW and Lauchpad
    input_lp = launch(input_trace, pad_width=launch_size[0], taper_length=launch_size[1], angle=0)
    input_lp_ref = twpa << input_lp
    input_lp_ref.dmove(input_loc)

    ## Input CPW
    input_trace.turn(radius=io_turn_radius, angle=-90)
    input_trace.straight(io_length)
    input_trace.turn(radius=turn_radius, angle=180)
    input_route = input_trace.make()
    input_route_ref = twpa << input_route
    input_route_ref.dmove(input_loc)

    ## Output CPW
    output_route_ref = twpa.add_ref(input_route)
    output_route_ref.dmove(input_loc).mirror_x()
    launch_out_ref = twpa.add_ref(input_lp)
    launch_out_ref.dmove(input_loc).mirror_x()
    if (M_rows%2): 
        output_route_ref.mirror_y()
        launch_out_ref.mirror_y()

    ## Turn CPWs
    chain_connect_cpw = gf.Component()
    connect_trace = Trace(width=input_trace.line_width, spacing=input_trace.line_spacing)
    connect_trace.turn(radius=turn_radius, angle=180)
    chain_connect_cpw.add_ref(connect_trace.make()).drotate(-90)
    chain_connect_cpw.add_ref(connect_trace.make()).drotate(90).dmovey(N_cells_per_row*2*cell_width)
    turns = twpa.add_ref(chain_connect_cpw, columns=M_rows/2, spacing=(2*(segment_spacing+cell_height), 0), rows=1)
    turns.dmove(chain_start)
    turns.dmovex((segment_spacing+cell_height))

    #Border for sawing
    saw_border = border(size=bottom_chip_size)

    # Chip assembly
    chip = gf.Component()
    chip.add_ref(twpa)
    chip.add_ref(saw_border)

    num_pads_per_row = math.floor((top_chip_height-2*margin)/2/cell_width)
    num_pad_rows = math.floor((top_chip_width-2*margin)/(cell_height+segment_spacing))

    # Solder bump array for flip chip design
    bottom_solder_array = solder_bump_array(bump_diameter=bump_diameter,
                                            underbump_metal_width=underbump_metal_width,
                                            cols=num_pad_rows+1, 
                                            rows=2*num_pads_per_row+1, 
                                            spacing=(cell_height+segment_spacing, cell_width), 
                                            centered=True)

    bottom_ti_array = titanium_array(metal_width=underbump_metal_width,
                                    cols=num_pad_rows+1, 
                                    rows=2*num_pads_per_row+2, 
                                    spacing=(cell_height+segment_spacing, cell_width), 
                                    centered=True)
    
    bottom_bump_array_ref = chip << bottom_solder_array
    bottom_ti_array_ref = chip << bottom_ti_array

    # Flipchip alignment markers
    alignment = flip_chip_alignment(size=(250,250), center_offset=(top_chip_size[0]-1150, top_chip_size[1]-750))
    bottom_alignment_ref = chip << alignment

    top_chip = gf.Component()
    top_border = border(size=top_chip_size)
    top_solder_array = solder_bump_array(bump_diameter=bump_diameter,
                                        underbump_metal_width=underbump_metal_width,
                                        cols=num_pad_rows+1, 
                                        rows=2*num_pads_per_row+1, 
                                        spacing=(cell_height+segment_spacing, cell_width), 
                                        centered=True,
                                        top_chip=True)
    
    top_ti_array = titanium_array(metal_width=underbump_metal_width,
                                    cols=num_pad_rows+1, 
                                    rows=2*num_pads_per_row+2, 
                                    spacing=(cell_height+segment_spacing, cell_width), 
                                    centered=True,
                                    top_chip=True)
    
    top_border_ref = top_chip << top_border
    top_bump_array_ref = top_chip << top_solder_array
    top_ti_array_ref = top_chip << top_ti_array
    top_alignment_ref = top_chip << alignment
    top_chip_ref = chip << top_chip
    # top_chip_ref.dmove((1.5*chip_width, chip_height/2))



    # Branding
    branding = gf.Component()
    ## Sample name
    name_cmp = gf.components.text_freetype(text='RevKerrTWPAI', size=100, justify='left', layer=layermap.SC1_E, font="Arial")
    name_ref = branding << name_cmp
    ## Lab name
    fab_cmp = gf.components.text_freetype(text="QNL", size=75, layer=layermap.SC1_E, font="Arial")
    fab_ref = branding << fab_cmp
    fab_ref.dmove((0, -75))
    ## AQT Logo
    aqt_logo = gf.read.import_gds(gdspath="../GDS_assets/AQT.gds", cellname='AQT').remap_layers(layer_map={(0, 0): (2, 0)})
    aqt_logo_ref = branding.add_ref(aqt_logo)
    aqt_logo_ref.dmove((950, 30))
    
    (x1,y1), (x2,y2) = branding.bbox_np()
    branding_keepout_ref = branding << gf.components.rectangle(size=(x2-x1,y2-y1), layer=layermap.KEEPOUT)
    branding_keepout_ref.dmove((x1, y1))
    branding_ref = chip << branding
    branding_ref.dmove((-(x2-x1)/2, chip_height/2-175))

    return chip

twpa = kerr_twpa(margin = 400, 
                segment_spacing=300, 
                launch_size=(125,225),
                num_fingers=0,
                finger_size=(0.1,0.1),
                pad_size=(50,30),
                taper_length=100,
                top_chip_size=(8500,8500)
                )
twpa.show()

/Users/elias/Desktop/School/Quantum/AQT/TWPA Project/qnlmodels/cpw.py:33: RuntimeWarning: divide by zero encountered in scalar divide
  K = sp.integrate.quad(lambda t: 1 / np.sqrt( (1 - t**2) * (1 - (k*t)**2) ), 0, 1)
/Users/elias/Desktop/School/Quantum/AQT/TWPA Project/qnlmodels/cpw.py:33: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
  K = sp.integrate.quad(lambda t: 1 / np.sqrt( (1 - t**2) * (1 - (k*t)**2) ), 0, 1)
/Users/elias/Desktop/School/Quantum/AQT/TWPA Project/qnlmodels/cpw.py:37: RuntimeWarning: divide by zero encountered in scalar divide
  return sp.integrate.quad(lambda t: 1 / np.sqrt( (1 - t**2) * (1 - (k*t)**2) ), 0, 1)[0]
/Users/elias/Desktop/School/Quantum/AQT/TWPA Project/qnlmodels/cpw.py:37: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
  return sp.integrate.quad(lambda t: 1 / np.sqrt( (1 - t**2) * (1 - (k*t)**2) ), 0, 1)[0]


2024-12-12 14:24:22.839 | WARNING  | gdsfactory.component:__getattribute__:138 - Getting `rectangle_S0p2_3_LJJ1_U_18d1912a_0_0.movex` in um is deprecated and will change to DataBaseUnits in gdsfactory9. Please use `rectangle_S0p2_3_LJJ1_U_18d1912a_0_0.dmovex` instead. For further information, please consult the migration guide: https://gdsfactory.github.io/gdsfactory/notebooks/21_migration_guide_7_8.html
2024-12-12 14:24:22.869 | WARNING  | gdsfactory.component:__getattribute__:138 - Getting `rectangle_S0p2_3_LJJ1_U_18d1912a_0_0.movex` in um is deprecated and will change to DataBaseUnits in gdsfactory9. Please use `rectangle_S0p2_3_LJJ1_U_18d1912a_0_0.dmovex` instead. For further information, please consult the migration guide: https://gdsfactory.github.io/gdsfactory/notebooks/21_migration_guide_7_8.html
2024-12-12 14:24:22.873 | WARNING  | gdsfactory.component:__getattribute__:138 - Getting `rectangle_S0p5_3_LJJ1_C_9c9e5ce6_0_0.movex` in um is deprecated and will change to DataBaseUni

/Users/elias/Desktop/School/Quantum/AQT/TWPA Project/qnlfactory/components.py:253: UserWarning: Getting `rectangle_S0p2_3_LJJ1_U_18d1912a_0_0.movex` in um is deprecated and will change to DataBaseUnits in gdsfactory9. Please use `rectangle_S0p2_3_LJJ1_U_18d1912a_0_0.dmovex` instead.
  large_junctions.add_ref(large_dolan_bridge).movex(-(large_bridge_gap+dolan_bridge_width)/2)
/Users/elias/Desktop/School/Quantum/AQT/TWPA Project/qnlfactory/components.py:254: UserWarning: Getting `rectangle_S0p2_3_LJJ1_U_18d1912a_0_0.movex` in um is deprecated and will change to DataBaseUnits in gdsfactory9. Please use `rectangle_S0p2_3_LJJ1_U_18d1912a_0_0.dmovex` instead.
  large_junctions.add_ref(large_dolan_bridge).movex(-(large_bridge_gap+dolan_bridge_width)/2).mirror_x()
/Users/elias/Desktop/School/Quantum/AQT/TWPA Project/qnlfactory/components.py:257: UserWarning: Getting `rectangle_S0p5_3_LJJ1_C_9c9e5ce6_0_0.movex` in um is deprecated and will change to DataBaseUnits in gdsfactory9. Please use `rec


2024-12-12 14:24:24.238 | INFO     | kfactory.kcell:show:8727 - klive v0.3.3: Opened file '/Users/elias/Desktop/School/Quantum/AQT/build/gds/2099377179.oas'


In [3]:
twpa.write_gds("RevKerrTWPA.gds")

PosixPath('RevKerrTWPA.gds')

In [4]:
### SCRAP CODE ###

# from qfab.components import (
#     airbridge, 
#     bond_pad,
#     border,
#     branding,
#     coupling_pad,
#     dipole_qubit,
#     id_box,
#     interdigitated_capacitor,
#     probe_alignment,
#     junction_lead,
#     junction_lead_v2,
#     manhattan_junction,
# )
# from qfab.device import (
#     flux_trap_layout,
#     standard_device_layout,
#     subtract_cutouts,
#     positive_mask,
# )
# from qfab.pdk import CPW
# from qfab.pdk import default_pdk as pdk
# from qfab.utils import meander, junction_cd_bias

# pdk.activate()



### TAPERING UTILS ###

# taper_length = (2*io_length+chain_length*(num_taper_paths-2))*1e-6
# f = mid_freq / 2 / np.pi
# wavelength = cpw_50_ohms.phase_velocity() / f
# B = 2 * np.pi / wavelength
# A = B * taper_length
# print("Taper length = {:.3f}".format(taper_length))
# print("A = {:.3f}".format(A))
# print("COSH(A) [dB] = {:.3f}".format(np.arccosh(10**(A/20))))
# taper = kt.KlopfensteinTaper(A)
# req_z0s = taper.get_all_impedances(50, cell_impedance)
# widths = np.array([cpw_50_ohms.solve_for_impedance(z0, 'w') for z0 in req_z0s])
# rev_widths = np.flip(widths)

# def taper_width_function(x, widths, spacing=0):
#     """ Args:
#     x : normalized N-vector (0 < x[i] < 1 for all i)
    
#     Returns: 
#     width : N-vector of widths
#     """  
#     width_array = []
#     N = len(x)
#     M = len(widths)
#     for i in range(N):
#         w = round_to_even_float(widths[int(i*M/N)]+spacing) if round(widths[int(i*M/N)]+spacing,3) % 2 else round(widths[int(i*M/N)]+spacing,3)
#         width_array = np.append(width_array, [w])
#     return width_array



### Old Cell and TWPA Layout

# cell_layout = EmbeddedSnail()
# # cell_layout.unit_cell().show()

# def kerr_twpa(bottom_chip_size: tuple=(10_000, 10_000), 
#               top_chip_size: tuple=(9_000,9_000), 
#               cell_impedance: float = 50, 
#               margin: float = 250,
#               segment_spacing: float = 150, 
#               taper_segment_spacing: float = 150, 
#               num_paths: int = 16, 
#               cell: gf.Component = None,
#               launch_size: tuple = (250, 250),
#               bump_diameter: float = 50,
#               underbump_metal_width: float = 80):
    
#     # Substrate parameters 
#     chip_width = bottom_chip_size[0]
#     chip_height = bottom_chip_size[1]
    
#     cell = cell
#     cell_height = cell.cell_height
#     cell_width = cell.cell_width

#     twpa = gf.Component()
#     metal = gf.components.rectangle(size=(chip_width, chip_height), layer=layermap.SC1_V, centered=True)

#     amp_cpw = CPW.CPW(cell.snail_gap_height, cell.finger_width, 675, 11.7)
#     amp_cpw = CPW.CPW( amp_cpw.solve_for_impedance(cell_impedance,'w'), cell.finger_width, 675, 11.7)
#     cpw_50_ohms = CPW.CPW(amp_cpw.solve_for_impedance(50,'w'), amp_cpw.solve_for_impedance(cell_impedance,'s'), 675, 11.7)

#     turn_radius = segment_spacing/2 + cell_width/2 + cell.finger_pad
#     io_turn_radius = taper_segment_spacing/2 + cpw_50_ohms.width/2

#     chain_length = chip_height-2*margin-2*turn_radius
#     io_length = chip_height/2 - margin - turn_radius - io_turn_radius

#     # [TAPERING UTILS WERE HERE] #
    
#     # Input geometry
#     ## Starting location
#     input_loc = (-chip_width/2+margin+launch_size[0]/2, 0)
    
#     ## Impedance matched trace
#     input_trace = Trace(width=cpw_50_ohms.w, spacing=cpw_50_ohms.s)
    
#     ## Lauchpad
#     input_lp = launch(input_trace, pad_width=launch_size[0], taper_length=launch_size[1], angle=0)
#     input_lp_ref = twpa << input_lp
#     input_lp_ref.dmove(input_loc)

#     ## CPW
#     input_trace.turn(radius=io_turn_radius, angle=-90)
#     input_trace.straight(io_length)
#     input_trace.turn(radius=turn_radius, angle=180)
#     input_route = input_trace.make()
#     input_route_ref = twpa << input_route
#     input_route_ref.dmove(input_loc)

#     # Cutout contians metal + etch space, used to get etch space later
#     cutout = gf.Component()
#     # Move start to begin TWPA chain layouts
#     chain_start = (input_loc[0]+2*turn_radius+io_turn_radius, -io_length-io_turn_radius)
#     ## Initial semicircle turn CPW
#     chain_connect_cpw = Trace(width=input_trace.line_width, spacing=3)
#     turn = twpa.add_ref(chain_connect_cpw.make()).drotate(90).dmove(chain_start)
#     ## Number of cells per chain
#     N_lin_cells = 2*math.floor(chain_length / 2 / cell_height)
#     ## For etch components
#     chain_cutout = gf.Component()
#     turn_cutout = gf.Component() 
    
#     for i in range(num_paths):
        
#         chain = twpa.add_ref(cell.chain_cells(N_lin_cells))
#         chain.connect("in", turn["out_w"], allow_width_mismatch=True)

#         chain_cutout.add_port(name="in", center=[cell_width/2, N_lin_cells*cell_height], width=cell.snail_gap_height, orientation=90, layer=layermap.SC1, port_type='electrical')
#         chain_cutout.add_port(name="out", center=[cell_width/2, 0], width=cell.snail_gap_height, orientation=-90, layer=layermap.SC1, port_type='electrical')
        
#         chain_cutout.add_polygon(gf.kdb.Region(cell.chain_cells(N_lin_cells).get_polygons()[1]).sized(cell.finger_pad*1e3), layer=layermap.SC1)
#         chain_cutout_ref = cutout.add_ref(chain_cutout)
#         chain_cutout_ref.connect("in", turn["out_w"], allow_width_mismatch=True)

#         chain_connect_cpw = Trace(width=input_trace.line_width, spacing=3)
#         chain_connect_cpw.turn(radius=turn_radius, angle=180*(-1)**(i+1))
#         turn = twpa.add_ref(chain_connect_cpw.make())
#         turn.connect("in_w", chain.ports["out"], allow_width_mismatch=True)

#         turn_cutout.add_port(name="in_w", center=[0, 0], width=cell.snail_gap_height, orientation=180, layer=layermap.SC1, port_type='electrical')
#         turn_cutout.add_port(name="out_w", center=[0, -2*turn_radius], width=cell.snail_gap_height, orientation=180, layer=layermap.SC1, port_type='electrical')
        
#         if i == 0:
#             turn_cutout.add_polygon(gf.kdb.Region(chain_connect_cpw.make().get_polygons()[1]).sized(input_trace.line_spacing*1e3), layer=layermap.SC1)
#         turn_cutout_ref = cutout.add_ref(turn_cutout)
#         if i % 2:
#             turn_cutout_ref.connect("out_w", chain.ports["out"], allow_width_mismatch=True)
#         else:
#             turn_cutout_ref.connect("in_w", chain.ports["out"], allow_width_mismatch=True)

#     print(f"{N_lin_cells*(num_paths-2)} cells") 
    
#     output_trace = Trace(width=cpw_50_ohms.w, spacing=cpw_50_ohms.s)
#     output_trace = output_trace.straight(io_length)
#     output_trace.turn(radius=io_turn_radius, angle=-90*(-1**(num_paths%2)))
#     cpw_out_route = output_trace.make()
#     cpw_out_route_ref = twpa << cpw_out_route

#     cpw_out_route_ref.connect("in_w", (turn_cutout_ref["out_w"] if num_paths % 2 else turn_cutout_ref["in_w"]), allow_width_mismatch=True)
    
#     launch_out = launch(output_trace, pad_width=launch_size[0], taper_length=launch_size[1], port="out_w")
#     launch_out_ref = twpa << launch_out
#     launch_out_ref.connect("cpw_side", cpw_out_route_ref["out_w"], allow_width_mismatch=True)

#     #Define etch component using cutout
#     etch = gf.Component()
#     etch.add_ref(gf.boolean(A=cutout, B=twpa, operation='not', layer1=layermap.SC1, layer2=layermap.SC1, layer=layermap.SC1_E))
    
#     #Border for sawing
#     saw_border = border(size=bottom_chip_size)

#     # Chip assembly
#     chip = gf.Component()
#     chip.add_ref(twpa)
#     chip.add_ref(saw_border)
#     chip.add_ref(etch)

#     # Solder bump array for flip chip design
    
#     solder_array = solder_bump_array(bump_diameter=bump_diameter,
#                                         underbump_metal_width=underbump_metal_width,
#                                         cols=num_paths+1, 
#                                         rows=N_lin_cells, 
#                                         spacing=(cell_width+segment_spacing+2*cell.finger_pad, cell_height), 
#                                         center_offset=(chain_start[0]-cell_width/2-segment_spacing/2-cell.finger_pad, chain_start[1]+cell_height/2))
#     chip.add_ref(solder_array)

#     # Branding
#     branding = gf.Component()
#     ## Sample name
#     name_cmp = gf.components.text_freetype(text='RevKerrTWPAI', size=100, justify='left', layer=layermap.SC1_E, font="Arial")
#     name_ref = branding << name_cmp
#     ## Lab name
#     fab_cmp = gf.components.text_freetype(text="QNL", size=75, layer=layermap.SC1_E, font="Arial")
#     fab_ref = branding << fab_cmp
#     fab_ref.dmove((0, -75))
#     ## AQT Logo
#     aqt_logo = gf.read.import_gds(gdspath="../GDS_assets/AQT.gds", cellname='AQT').remap_layers(layer_map={(0, 0): (2, 0)})
#     aqt_logo_ref = branding.add_ref(aqt_logo)
#     aqt_logo_ref.dmove((950, 30))
    
#     (x1,y1), (x2,y2) = branding.bbox_np()
#     branding_keepout_ref = branding << gf.components.rectangle(size=(x2-x1,y2-y1), layer=layermap.KEEPOUT)
#     branding_keepout_ref.dmove((x1, y1))
#     branding_ref = chip << branding
#     branding_ref.dmove((-(x2-x1)/2, chip_height/2-175))

#     top_chip = gf.Component()
#     top_border = border(size=top_chip_size)
#     bump_array = solder_bump_array(bump_diameter=bump_diameter,
#                                     underbump_metal_width=underbump_metal_width,
#                                     cols=num_paths+1, 
#                                     rows=N_lin_cells, 
#                                     spacing=(cell_width+segment_spacing+2*cell.finger_pad, cell_height), 
#                                     top_chip=True, 
#                                     center_offset=(chain_start[0]-cell_width/2-segment_spacing/2-cell.finger_pad, chain_start[1]+cell_height/2))

#     border_ref = top_chip << top_border
#     bump_array_ref = top_chip << bump_array
#     top_chip_ref = chip << top_chip
#     # top_chip_ref.dmove((1.5*chip_width, chip_height/2))

#     flip_chip_align = flip_chip_alignment(center_offset=top_chip_size)
#     flip_chip_align_ref = chip << flip_chip_align

#     return chip


# twpa = kerr_twpa(margin = 400, 
#                 segment_spacing=150, 
#                 cell=cell_layout, 
#                 launch_size=(125,225),
#                 num_paths=21, 
#                 taper_segment_spacing = 150)
# twpa.show()